Specs for the notebook:
- ideally takes any huggingface model but llama of some sort is good
- produces the difference vectors
- produces intervened vectors for entities/attributes
- produces the corresponding native vectors for the entities/attributes

In [14]:
import torch
import random

import pandas as pd

from transformers import AutoTokenizer, AutoModelForCausalLM

def load_model_and_tokenizer(gpu_num, model_name="meta-llama/Llama-2-7b-chat-hf"):
    device = torch.device(f"cuda:{gpu_num}")
    model = AutoModelForCausalLM.from_pretrained(
        model_name, cache_dir="/data2/sjeromeh/cache/pretrained_models"
    ).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="/data2/sjeromeh/cache/pretrained_models")
    tokenizer.pad_token_id = tokenizer.eos_token_id
    model.eval()
    return model, tokenizer

Load model and tokenizer

In [ ]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
model, tokenizer = load_model_and_tokenizer(0, model_name=model_name)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.00s/it]


Load data

In [15]:
countries_df = pd.read_csv("data/countries.csv")
names_df = pd.read_csv("data/names.csv", index_col=0)

countries_to_capitals = countries_df.set_index("country")["capital"].to_dict()
names = names_df["name"].tolist()

print(f"Number of countries: {len(countries_to_capitals)}, Sample: {list(countries_to_capitals.items())[:5]}")
print(f"Number of names: {len(names)}, Sample: {names[:5]}")

Number of countries: 247, Sample: [('Abkhazia', 'Sukhumi'), ('Afghanistan', 'Kabul'), ('Akrotiri and Dhekelia', 'Episkopi Cantonment'), ('Albania', 'Tirana'), ('Algeria', 'Algiers')]
Number of names: 100, Sample: ['Michael', 'James', 'John', 'Robert', 'David']


Prompt generation

In [27]:
def generate_prompts(num_entities):
    cs = random.sample(list(countries_to_capitals.keys()), num_entities*2)
    ns = random.sample(names, num_entities*2)

    cs_1 = cs[:num_entities]
    ns_1 = ns[:num_entities]
    cs_2 = cs[num_entities:]
    ns_2 = ns[num_entities:]

    es_1 = [f"{n} lives in the capital city of {c}." for c, n in zip(cs_1, ns_1)]
    es_2 = [f"{n} lives in the capital city of {c}." for c, n in zip(cs_2, ns_2)]
    context =  f"Context: {' '.join(es_1)}"
    alt_context =  f"Context: {' '.join(es_2)}"
    questions = [f"Question: Which city does {n} live in?" for n in ns_1]
    answers = [countries_to_capitals[c] for c in cs_1]
    alt_answers = [countries_to_capitals[c] for c in cs_2]
    return context, alt_context, questions, answers, alt_answers

context, alt_context, questions, answers, alt_answers = generate_prompts(5)
print(context)
print(alt_context)
print(questions)
print(answers)

Context: Stephen lives in the capital city of Western Sahara. Rebecca lives in the capital city of Ecuador. Rachel lives in the capital city of Iceland. Timothy lives in the capital city of Seychelles. Zachary lives in the capital city of Bangladesh.
Context: Brenda lives in the capital city of Cocos (Keeling) Islands. Paul lives in the capital city of Syria. Patrick lives in the capital city of Barbados. Ronald lives in the capital city of Kuwait. Benjamin lives in the capital city of Azerbaijan.
['Question: Which city does Stephen live in?', 'Question: Which city does Rebecca live in?', 'Question: Which city does Rachel live in?', 'Question: Which city does Timothy live in?', 'Question: Which city does Zachary live in?']
['El Aaiún', 'Quito', 'Reykjavík', 'Victoria', 'Dhaka']
